In [63]:
print("Hello")

Hello


In [70]:
import os
os.chdir(r"D:\AI\Medi__Chat")

In [71]:
%pwd

'D:\\AI\\Medi__Chat'

In [72]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [73]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="**/*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [74]:
extracted_data = load_pdf(data ='Data/')

In [ ]:
extracted_data

In [75]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunks = text_splitter.split_documents(extracted_data)
    return texts_chunks

In [76]:
texts_chunks = text_split(extracted_data)
print(f"Total number of chunks: {len(texts_chunks)}")

Total number of chunks: 41


In [77]:
from langchain.embeddings import HuggingFaceEmbeddings

In [78]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [80]:
embeddings= download_huggingface_embeddings()

In [81]:
query_result = embeddings.embed_query("What is the capital of France?")
print("Query result:" ,len(query_result)) 

Query result: 384


In [ ]:
query_result


In [83]:
from dotenv import load_dotenv
load_dotenv()

True

In [84]:
import os
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")


In [85]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [ ]:

import os
from pinecone import Pinecone, ServerlessSpec

api_key = os.environ.get("PINECONE_API_KEY")

if not api_key:
    raise ValueError("PINECONE_API_KEY not found in environment variables.")
pc = Pinecone(api_key=api_key)

index_name = "medichat"

if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"  
        )
    )

print(f"Index '{index_name}' is ready!")

In [87]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunks,
    index_name='medichat',
    embedding=embeddings,
)

In [88]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name='medichat',
    embedding=embeddings,
)

In [89]:
docsearch

In [90]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [91]:
retrived_docs = retriver.invoke("What is the Napa?")

In [23]:
retrived_docs

[Document(id='308e22bf-cb35-4a0d-887c-cc99f6e118c0', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 0.0, 'page_label': '1', 'producer': 'Skia/PDF m138 Google Docs Renderer', 'source': 'Data\\napa.pdf', 'title': 'Untitled document', 'total_pages': 9.0}, page_content="Napa  Tablet  \n Pack  Image \nParacetamol \n500  mg  \nBeximco  Pharmaceuticals  Ltd. \nUnit  Price:  ৳  1.20  (51  x  10:  ৳  612.00)  \nStrip  Price: ৳  12.00  \nAlso  available  as:  \n665  mg  (ER  Tablet) 500  mg  (Tablet) 1000  mg  (Tablet) \nAlternate  Brands Innovator's  Monograph     বাংলায়  \x00দ খ ু ন  \nIndications  \nNapa  is  indicated  for  fever,  common  cold  and  influenza,  headache,  toothache,  earache,  bodyache,  \nmyalgia,\n \nneuralgia,\n \ndysmenorrhoea,\n \nsprains,\n \ncolic\n \npain,"),
 Document(id='d4462d89-8573-4180-89d6-ffb545970cdc', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 0.0, 'page_label': '1', 'producer': 'Skia/PDF m138 Google Docs Renderer', 'source': 'Dat

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",                    
    temperature=0,
    max_output_tokens=500,
    google_api_key=GEMINI_API_KEY 
)

In [93]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful medical assistant. "
    "You will be provided with a question and some context from medical documents. "
    "Use the context to answer the question as accurately as possible."
    "{context}\n"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [94]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


question_answer_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

rag_chain = create_retrieval_chain(
    retriever=retriver,
    combine_docs_chain=question_answer_chain,
)


In [95]:

import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

API_KEY = GEMINI_API_KEY

print("Testing API Key...")
print(f"API Key length: {len(API_KEY)}")
print(f"API Key starts with 'AIza': {API_KEY.startswith('AIza')}")
try:
    genai.configure(api_key=API_KEY)
    print("\n=== Testing with google.generativeai ===")
    models = list(genai.list_models())
    print(f"Successfully connected! Found {len(models)} models")
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content("Hello, world!")
    print(f"Test response: {response.text[:50]}...")
    
except Exception as e:
    print(f"Direct API test failed: {e}")
try:
    print("\n=== Testing with LangChain ===")
    

    os.environ["GOOGLE_API_KEY"] = API_KEY
    
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0
    )
    
    response = llm.invoke("Say hello")
    print(f"LangChain test successful: {response.content[:50]}...")
    
except Exception as e:
    print(f"LangChain test failed: {e}")
    
    try:
        print("Trying direct API key parameter...")
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            google_api_key=API_KEY
        )
        
        response = llm.invoke("Say hello")
        print(f"Direct API key test successful: {response.content[:50]}...")
        
    except Exception as e2:
        print(f"Direct API key test also failed: {e2}")

print("\n=== Debugging Info ===")
print(f"Environment GOOGLE_API_KEY set: {'GOOGLE_API_KEY' in os.environ}")
if 'GOOGLE_API_KEY' in os.environ:
    env_key = os.environ['GOOGLE_API_KEY']
    print(f"Environment key length: {len(env_key)}")
    print(f"Environment key starts with 'AIza': {env_key.startswith('AIza')}")

Testing API Key...
API Key length: 39
API Key starts with 'AIza': True

=== Testing with google.generativeai ===
Successfully connected! Found 58 models
Test response: Hello, world!  (back at you!)
...

=== Testing with LangChain ===
LangChain test successful: Hello!...

=== Debugging Info ===
Environment GOOGLE_API_KEY set: True
Environment key length: 39
Environment key starts with 'AIza': True


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=API_KEY  
)

response = llm.invoke("Say hello")
print(f"LangChain test successful: {response.content}")


LangChain test successful: Hello!


In [ ]:
import time


time.sleep(1)  
response = rag_chain.invoke({"input": "Napa"})

response_text = response['answer']


prefix = "Based on the provided text, "
if response_text.startswith(prefix):
    response_text = response_text[len(prefix):]

print(response_text)


Napa is a brand of paracetamol tablets manufactured by Beximco Pharmaceuticals Ltd.  It's available in 500mg, 665mg (extended release), and 1000mg strengths.  The 500mg tablets are priced at ৳1.20 per unit, or ৳12.00 per strip of ten.  Napa is indicated for the treatment of fever, common cold and influenza, headache, toothache, earache, bodyache, myalgia, neuralgia, dysmenorrhea, sprains, and colic pain.
